# $p$-values for Standarised $\beta$'s, partial correlation coefficient, and semi-partial correlation coefficient

In [2]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

from pingouin import partial_corr

Import `duncan_prestige` dataset:

In [10]:
duncan_prestige = sm.datasets.get_rdataset("Duncan", "carData").data.drop("type", axis=1)

# standardise Y and X
Y = (duncan_prestige['income'] - duncan_prestige['income'].mean()) / duncan_prestige['income'].std()
X = (duncan_prestige[['education', 'prestige']] - duncan_prestige[['education', 'prestige']].mean()) / duncan_prestige[['education', 'prestige']].std()

# perform regression
model = sm.OLS(Y,X)
results = model.fit()

In [11]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 income   R-squared (uncentered):                   0.702
Model:                            OLS   Adj. R-squared (uncentered):              0.688
Method:                 Least Squares   F-statistic:                              50.73
Date:                Fri, 28 May 2021   Prob (F-statistic):                    4.84e-12
Time:                        14:09:05   Log-Likelihood:                         -36.081
No. Observations:                  45   AIC:                                      76.16
Df Residuals:                      43   BIC:                                      79.78
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
education      0.0393      0.159      0.247      0.806      -0.281       0.360
prestige       0.8043      0.159      5.063      0.000       0.484       1.125
==============================================================================
Omnibus:                        9.200   Durbin-Watson:                   2.053
Prob(Omnibus):                  0.010   Jarque-Bera (JB):               21.265
Skew:                           0.075   Prob(JB):                     2.41e-05
Kurtosis:                       6.364   Cond. No.                         3.54
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Compute partial correlation coefficients:

In [15]:
partial_corr_dict = {}

for predictor in ["education", "prestige"]:
    
    partial_corr_dict[predictor] = partial_corr(data=duncan_prestige,
                                                y="income",
                                                x=predictor,
                                                covar=duncan_prestige.columns.drop([predictor, "income"]).to_list()
                                                )[["r", "p-val"]].reset_index().drop("index", axis=1).values[0]
    

partial_corr_df = pd.DataFrame.from_dict(partial_corr_dict,
                                         orient="index",
                                         columns=["r", "p_val"]
                                        ).sort_values(
    "r", key=abs, ascending=False)

In [26]:
partial_corr_df

,r,p_val
prestige,0.611101,0.000011
education,0.037690,0.808084


We see that the $p$-values are similar (probably small calculation difference/numerical error) to the regression results above.

We now consider the semi-partial correlation coefficient:

In [29]:
semi_partial_corr_dict = {}

for predictor in ["education", "prestige"]:
    
    semi_partial_corr_dict[predictor] = partial_corr(data=duncan_prestige,
                                                     y="income",
                                                     x=predictor,
                                                     y_covar=duncan_prestige.columns.drop([predictor, "income"]).to_list()
                                                     )[["r", "p-val"]].reset_index().drop("index", axis=1).values[0]
    

semi_partial_corr_df = pd.DataFrame.from_dict(semi_partial_corr_dict,
                                              orient="index",
                                              columns=["r", "p_val"]
                                              ).sort_values(
    "r", key=abs, ascending=False)

In [30]:
semi_partial_corr_df

,r,p_val
prestige,0.320021,0.034200
education,0.019738,0.898807


These are noticeably different (larger) than those seen in the OLS.